In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import check_array, check_X_y
from keras import regularizers
pd.options.mode.chained_assignment = None

In [2]:
data = pd.read_table("sample_data.txt")
data = data.iloc[:, :11]
dv = data.iloc[:, 10:11]
features = data.iloc[:, :10]

In [3]:
#function that takes in a features pandas dataframe and turns it into a numpy array 
#with only the categorical variables one-hot encoded (and leaving out one of the features 
#of every one-hot encoded variable as baseline)
#@param:  features is a pandas df of features; threshold number is the number of unique values 
#a variable should have in order to be considered categorical
#@return:  a numpy matrix containing the original dataset but with the categorical variables
#one hot encoded and one of the one hot encoded features per cat variable left out; 
#for example an original feature set of 10 categorical variables with three categories each will 
#be transoformed into a numpy array with 20 dichotomous variables

def one_hot_encode(features, cat_threshold_number = 5):
    num_unique_vals_dict = {}
    feature_names = list(features)
    for feature in feature_names:
        label_encoder = LabelEncoder()
        features.loc[:, feature] = label_encoder.fit_transform(features.loc[:, feature])
        num_unique_vals_dict[feature] = len(label_encoder.classes_)

    features_to_onehot = []
    for feature in num_unique_vals_dict:
        if num_unique_vals_dict[feature] <= cat_threshold_number and num_unique_vals_dict[feature] > 1:
            features_to_onehot = features_to_onehot + [feature]

    #create index array listing indices in orginal feature names array that are present in features_to_onehot
    indices_to_onehot = np.nonzero(np.in1d(feature_names, features_to_onehot))

    onehot_encoder = OneHotEncoder(categorical_features = indices_to_onehot, sparse = False)
    features = onehot_encoder.fit_transform(features)

    idx_to_delete = np.cumsum([0] + list(num_unique_vals_dict.values()))

    idx_to_keep = [i for i in range(features.shape[1]) if i not in idx_to_delete]

    features = features[:, idx_to_keep]

    return features

In [4]:
#one hot encode data, then split it into training and validation
X = one_hot_encode(features)
y = dv
#split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.33, random_state = 42)
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_val = x_val.reshape((len(x_val)), np.prod(x_val.shape[1:]))

In [27]:
class SparseAutoencoder(object):
    
    def __init__(self,encoding_dim, regularizer, activation, optimizer, loss, epochs, batch_size, random_state = 42):
        self.regularizer = regularizer
        self.encoding_dim = encoding_dim
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss
        self.epochs = epochs
        self.batch_size = batch_size
        self.random_state = random_state
            
        
        
    def fit(self, X, y=None):
        """Do nothing and return the estimator unchanged
        This method is just there to implement the usual API and hence
        work in pipelines.
        Parameters
        ----------
        X : array-like
        """
        
        X, y = check_X_y(X, y, accept_sparse=True, dtype=None)
        x_train, x_val, _, _ = train_test_split(
            X, y, test_size=0.25, train_size=0.75, random_state=self.random_state)

        X = check_array(X, accept_sparse='csr')
        
        X_width = x_train.shape[1]
        
        self.input_placeholder = Input(shape = (X_width, ))    #input placeholder
        self.encoded = Dense(self.encoding_dim, activation = self.activation, 
                            activity_regularizer = self.regularizer)(self.input_placeholder)   #the encoded representation of input
        self.decoded = Dense(X_width, activation = 'sigmoid')(self.encoded)
        
        #define autoencoder model object
        self.autoencoder = Model(self.input_placeholder, self.decoded)
        #compile autoencoder model
        self.autoencoder.compile(optimizer = self.optimizer, loss = self.loss)
        self.autoencoder.fit(x_train, x_train, 
                             epochs = self.epochs, 
                             batch_size = self.batch_size, 
                             shuffle = True, 
                             validation_data = (x_val, x_val))
        
        #define separate encoder model object
        self.encoder = Model(self.input_placeholder, self.encoded)  
        return self
    
    
    def transform(self, X):
        
        X = check_array(X, accept_sparse='csr')
        encoded_preds = self.encoder.predict(X)
        return encoded_preds
    
  

In [28]:
sparse_ae = SparseAutoencoder(encoding_dim=40, 
                        regularizer = regularizers.l2(10e-6),
                        activation='relu', 
                        optimizer='adadelta', 
                        loss='binary_crossentropy', 
                        epochs=100, batch_size=200)



In [29]:
sparse_ae.fit(x_train, y_train)

C:\Users\monicai\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train on 502 samples, validate on 168 samples
Epoch 1/100
502/502 [==============================] - 0s 735us/step - loss: 0.7080 - val_loss: 0.7002
Epoch 2/100
502/502 [==============================] - 0s 22us/step - loss: 0.7009 - val_loss: 0.6936
Epoch 3/100
502/502 [==============================] - 0s 30us/step - loss: 0.6944 - val_loss: 0.6874
Epoch 4/100
502/502 [==============================] - 0s 32us/step - loss: 0.6881 - val_loss: 0.6814
Epoch 5/100
502/502 [==============================] - 0s 26us/step - loss: 0.6821 - val_loss: 0.6756
Epoch 6/100
502/502 [==============================] - 0s 22us/step - loss: 0.6762 - val_loss: 0.6699
Epoch 7/100
502/502 [==============================] - 0s 18us/step - loss: 0.6705 - val_loss: 0.6643
Epoch 8/100
502/502 [==============================] - 0s 22us/step - loss: 0.6649 - val_loss: 0.6588
Epoch 9/100
502/502 [==============================] - 0s 26us/step - loss: 0.6591 - val_loss: 0.6531
Epoch 10/100
502/502 [=============

502/502 [==============================] - 0s 18us/step - loss: 0.2849 - val_loss: 0.2885
Epoch 80/100
502/502 [==============================] - 0s 19us/step - loss: 0.2816 - val_loss: 0.2854
Epoch 81/100
502/502 [==============================] - 0s 20us/step - loss: 0.2785 - val_loss: 0.2823
Epoch 82/100
502/502 [==============================] - 0s 20us/step - loss: 0.2755 - val_loss: 0.2793
Epoch 83/100
502/502 [==============================] - 0s 22us/step - loss: 0.2724 - val_loss: 0.2763
Epoch 84/100
502/502 [==============================] - 0s 24us/step - loss: 0.2696 - val_loss: 0.2735
Epoch 85/100
502/502 [==============================] - 0s 22us/step - loss: 0.2669 - val_loss: 0.2707
Epoch 86/100
502/502 [==============================] - 0s 20us/step - loss: 0.2639 - val_loss: 0.2680
Epoch 87/100
502/502 [==============================] - 0s 18us/step - loss: 0.2614 - val_loss: 0.2652
Epoch 88/100
502/502 [==============================] - 0s 18us/step - loss: 0.2586 - 

In [31]:
sparse_ae.transform(x_train).shape

(670, 40)